# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [6]:
soup

<!DOCTYPE html>
<html lang="zh_TW">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="501887343352051" property="fb:app_id"/>
<meta content="Yahoo奇摩電影" property="og:site_name"/>
<link href="/favicon.ico?v=2" rel="icon"/>
<title>Yahoo奇摩電影</title>
<meta content="Yahoo奇摩電影" name="twitter:title"/>
<meta content="Yahoo奇摩電影" property="og:title"/>
<link href="https://movies.yahoo.com.tw" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw" property="og:url"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="twitter:description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" property="og:description"/>
<meta content="https://movies.yahoo.com.tw/build/images/ogImage.png" name="twitte

In [5]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 巴黎夢想家, ID: 12525
Movie: 通靈女校, ID: 12455
Movie: 生命理賠師, ID: 12454
Movie: 平行母親, ID: 12208
Movie: 蝙蝠俠, ID: 10904
Movie: 瑪雅蜜蜂大冒險：守護小金蛋, ID: 12605
Movie: 我是你的完美男友, ID: 12535
Movie: 密室譯戰, ID: 12453
Movie: 黑光行動, ID: 12444
Movie: 時代革命, ID: 12424
Movie: 瘋狂競賽片, ID: 12380
Movie: 櫻櫻大日本, ID: 12288
Movie: 春光乍洩 4K修復, ID: 12128
Movie: 情聖西哈諾, ID: 11553
Movie: 劇場版 咒術迴戰 0, ID: 12385
Movie: 黑盒線索, ID: 12433
Movie: 世界上最爛的人, ID: 12405
Movie: 鬼電梯, ID: 12389
Movie: 電影版 昨日的美食, ID: 12388
Movie: 垃圾頭, ID: 12322
Movie: 逍遙上路, ID: 12130
Movie: 完美的敵人, ID: 12050
Movie: 愛別離苦, ID: 11487
Movie: 秘境探險, ID: 11588
Movie: 王牌冤家, ID: 12534
Movie: 在車上, ID: 12274
Movie: 讓我聽見愛的歌聲, ID: 12197
Movie: 娶我吧！, ID: 11755
Movie: 尼羅河謀殺案, ID: 10897
Movie: 記憶, ID: 12442
Movie: 月球隕落, ID: 11507
Movie: 我吃了那男孩一整年的早餐, ID: 12202
Movie: 諸葛四郎－英雄的英雄, ID: 11748
Movie: 素還真, ID: 11545
Movie: 夜路, ID: 11526
Movie: 歡樂好聲音2, ID: 11413
Movie: 妖果小學-水果奶奶的大秘密, ID: 12194
Movie: 我的神秘狼朋友, ID: 12138
Movie: 屁屁偵探：舒芙蕾島的秘密, ID: 11993
Movie: G風暴, ID: 12041
Movie: 致命最

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [7]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 12385

In [9]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 南投, 代號(area_id): 13
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14
放映地區: 基隆, 代號(area_id): 18
放映地區: 宜蘭, 代號(area_id): 11
放映地區: 花蓮, 代號(area_id): 12
放映地區: 台東, 代號(area_id): 9
放映地區: 金門, 代號(area_id): 24
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [10]:
# 指定放映地區
area_id = 28

In [11]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

三月 9
三月 10
三月 11
三月 12
三月 13


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [13]:
# 選定要觀看的日期
date = "2022-03-09"

In [14]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [65]:
html

[<ul class="area_time _c jq_area_time" data-theater_name="欣欣秀泰影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=30" data-theater_url="http://www.showtimes.com.tw/" id="theater_id_30">
 <li class="adds">
 <a href="https://movies.yahoo.com.tw/theater_result.html/id=30">欣欣秀泰影城</a>
 <span>02-25371889</span>
 </li>
 <li class="taps">
 <span class="tapR">數位</span> </li>
 <li class="time _c">
 <div class="input_picker jq_input_picker">
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '劇場版 咒術迴戰 0']" data-movie_date="03.09" data-movie_time="10:30" data-movie_title="劇場版 咒術迴戰 0" data-movie_type="數位" data-news="news" id="77556070" name="schedule_list" type="radio" value="2022-03-09 10:30:00"/>
 <label class="" for="77556070">10:30</label>
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '劇場版 咒術迴戰 0']" data-movie_date="03.09" data-movie_time="11:30" data-movie_title="劇場版 咒術迴戰 0" data-movie_type="數位" data-news="news" id="77556071" name="schedule_list" typ

In [86]:
for i in html:
    print('電影院: ', i.find('a').text)
    print('放映類型: ', i.find('span',class_="tapR").text)
    for f in i.find_all('input'):
        print('放映時間:',f['value'])
    print('\n')

電影院:  欣欣秀泰影城
放映類型:  數位
放映時間: 2022-03-09 10:30:00
放映時間: 2022-03-09 11:30:00
放映時間: 2022-03-09 12:40:00
放映時間: 2022-03-09 13:40:00
放映時間: 2022-03-09 14:50:00
放映時間: 2022-03-09 15:50:00
放映時間: 2022-03-09 17:00:00
放映時間: 2022-03-09 18:00:00
放映時間: 2022-03-09 19:10:00
放映時間: 2022-03-09 21:20:00
放映時間: 2022-03-09 23:30:00
放映時間: 2022-03-10 00:30:00


電影院:  台北美麗華大直影城
放映類型:  數位
放映時間: 2022-03-09 16:30:00
放映時間: 2022-03-09 17:30:00
放映時間: 2022-03-09 18:40:00
放映時間: 2022-03-09 19:40:00
放映時間: 2022-03-09 20:20:00
放映時間: 2022-03-09 21:50:00
放映時間: 2022-03-09 22:30:00
放映時間: 2022-03-09 23:10:00
放映時間: 2022-03-10 00:00:00


電影院:  台北新光影城
放映類型:  數位
放映時間: 2022-03-09 17:15:00
放映時間: 2022-03-09 19:20:00


電影院:  台北in89豪華數位影城
放映類型:  數位
放映時間: 2022-03-09 16:10:00
放映時間: 2022-03-09 16:50:00
放映時間: 2022-03-09 18:15:00
放映時間: 2022-03-09 18:55:00
放映時間: 2022-03-09 20:20:00
放映時間: 2022-03-09 21:00:00
放映時間: 2022-03-09 22:25:00
放映時間: 2022-03-09 23:05:00


電影院:  台北樂聲影城
放映類型:  數位
放映時間: 2022-03-09 16:50:00
放映時間: 2022-03-09 17:55:00
放映時間: 2022